In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
from scipy.stats import iqr 
from sklearn.preprocessing import LabelEncoder
from collections import Counter as c
from sklearn.metrics import r2_score,mean_absolute_error,confusion_matrix, classification_report  
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('Car details v3.csv')
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [3]:
df=df.dropna()
df.isnull().sum()


name             0
year             0
selling_price    0
km_driven        0
fuel             0
seller_type      0
transmission     0
owner            0
mileage          0
engine           0
max_power        0
torque           0
seats            0
dtype: int64

In [4]:
u = pd.DataFrame(df['mileage'])
u

,mileage
0,23.4 kmpl
1,21.14 kmpl
2,17.7 kmpl
3,23.0 kmpl
4,16.1 kmpl
...,...
8123,18.5 kmpl
8124,16.8 kmpl
8125,19.3 kmpl
8126,23.57 kmpl


In [5]:
uu=u.to_numpy()
uu

array([['23.4 kmpl'],
       ['21.14 kmpl'],
       ['17.7 kmpl'],
       ...,
       ['19.3 kmpl'],
       ['23.57 kmpl'],
       ['23.57 kmpl']], dtype=object)

In [6]:
a=np.empty(len(uu))
for i in range(len(uu)):
    d=uu[i][0]
    if d[-5:]==' kmpl':
        x=d.strip(' kmpl')
    elif d[-6:]==' km/kg':
        x=d.strip(' km/kg')
    a[i]=x
print(a)
aa=a[:,np.newaxis]

[23.4  21.14 17.7  ... 19.3  23.57 23.57]


In [7]:
v = pd.DataFrame(df['engine'])
v

,engine
0,1248 CC
1,1498 CC
2,1497 CC
3,1396 CC
4,1298 CC
...,...
8123,1197 CC
8124,1493 CC
8125,1248 CC
8126,1396 CC


In [8]:
vv=v.to_numpy()
vv

array([['1248 CC'],
       ['1498 CC'],
       ['1497 CC'],
       ...,
       ['1248 CC'],
       ['1396 CC'],
       ['1396 CC']], dtype=object)

In [9]:
b=np.empty(len(vv),dtype='int')
for i in range(len(vv)):
    d=vv[i][0]
    if d[-3:]==' CC':
        x=d.strip(' CC')
    b[i]=x
print(b)
bb=b[:,np.newaxis]

[1248 1498 1497 ... 1248 1396 1396]


In [10]:
w=pd.DataFrame(df['max_power'])
w

,max_power
0,74 bhp
1,103.52 bhp
2,78 bhp
3,90 bhp
4,88.2 bhp
...,...
8123,82.85 bhp
8124,110 bhp
8125,73.9 bhp
8126,70 bhp


In [11]:
ww=w.to_numpy()
ww

array([['74 bhp'],
       ['103.52 bhp'],
       ['78 bhp'],
       ...,
       ['73.9 bhp'],
       ['70 bhp'],
       ['70 bhp']], dtype=object)

In [12]:
c=np.empty(len(ww))
for i in range(len(ww)):
    d=ww[i][0]
    if d[-4:]==' bhp':
        x=d.strip(' bhp')
    c[i]=x
print(c)
cc=c[:,np.newaxis]
cc.shape

[ 74.   103.52  78.   ...  73.9   70.    70.  ]


(7906, 1)

In [13]:
df=df.drop(['mileage','engine','max_power','torque'], axis=1)
df

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,5.0
...,...,...,...,...,...,...,...,...,...
8123,Hyundai i20 Magna,2013,320000,110000,Petrol,Individual,Manual,First Owner,5.0
8124,Hyundai Verna CRDi SX,2007,135000,119000,Diesel,Individual,Manual,Fourth & Above Owner,5.0
8125,Maruti Swift Dzire ZDi,2009,382000,120000,Diesel,Individual,Manual,First Owner,5.0
8126,Tata Indigo CR4,2013,290000,25000,Diesel,Individual,Manual,First Owner,5.0


In [14]:
zz=df.to_numpy()
zz.shape

(7906, 9)

In [15]:
r=np.concatenate((zz,aa,bb,cc),axis=1)
r

array([['Maruti Swift Dzire VDI', 2014, 450000, ..., 23.4, 1248, 74.0],
       ['Skoda Rapid 1.5 TDI Ambition', 2014, 370000, ..., 21.14, 1498,
        103.52],
       ['Honda City 2017-2020 EXi', 2006, 158000, ..., 17.7, 1497, 78.0],
       ...,
       ['Maruti Swift Dzire ZDi', 2009, 382000, ..., 19.3, 1248, 73.9],
       ['Tata Indigo CR4', 2013, 290000, ..., 23.57, 1396, 70.0],
       ['Tata Indigo CR4', 2013, 290000, ..., 23.57, 1396, 70.0]],
      dtype=object)

In [16]:
r.shape

(7906, 12)

In [18]:
dd = pd.DataFrame(data=r, index=[], columns=["nameyear","selling_price","km_driven","fuel","seller_type","transmission","owner","seats","mileage","engine","max_power"])

ValueError: Shape of passed values is (7906, 12), indices imply (0, 11)